<a href="https://colab.research.google.com/github/svarunkishore/Adaptive_Cyber_Defence_System/blob/main/Machine_Learning_Model_(Complete).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 7.2 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import shap

def train_and_explain_model(dataset_path, model_save_path):
    # Load your dataset
    dataset_df = pd.read_csv(dataset_path)

    # Preprocess your data and split into features (X) and labels (Y)
    Y = dataset_df['class']
    X = dataset_df.drop(['class'], axis=1)

    encoder = LabelEncoder().fit(Y)
    Y = encoder.transform(Y)

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    features = X.columns
    num_features = len(features)

    # Define and train your Keras model using the TensorFlow backend
    AN = tf.keras.Sequential()
    AN.add(tf.keras.layers.Dense(256, activation='relu', input_dim=num_features))
    AN.add(tf.keras.layers.Dropout(0.2))
    AN.add(tf.keras.layers.Dense(128, activation='relu'))
    AN.add(tf.keras.layers.Dropout(0.2))
    AN.add(tf.keras.layers.Dense(128, activation='relu'))
    AN.add(tf.keras.layers.Dropout(0.2))
    AN.add(tf.keras.layers.Dense(32, activation='relu'))
    AN.add(tf.keras.layers.Dropout(0.2))
    AN.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    AN.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
    AN.fit(X_train, y_train, epochs=100, batch_size=32)

    # Save the trained model
    AN.save(model_save_path)

    # Create an explainer for your Keras model
    explainer = shap.Explainer(AN, X_train.to_numpy())

    # Calculate and print the classification report
    y_pred = AN.predict(X_test)
    y_pred_binary = (y_pred > 0.5).astype(int)
    report = classification_report(y_test, y_pred_binary, target_names=['benign', 'malign'])
    print(report)

    # Calculate SHAP values for the test set
    shap_values = explainer.shap_values(X_test)

    # Calculate the mean importance of each feature across all instances
    abs_shap_values = np.abs(shap_values)
    mean_abs_shap = np.mean(abs_shap_values, axis=0)

    # Create a DataFrame to store feature names and their mean SHAP values
    feature_importance_df = pd.DataFrame(list(zip(features, mean_abs_shap)), columns=['Feature', 'SHAP Value'])
    feature_importance_df = feature_importance_df.sort_values(by='SHAP Value', ascending=False)

    # Extract the top feature with the highest SHAP value
    top_feature = feature_importance_df.iloc[0]['Feature']
    top_shap_value = feature_importance_df.iloc[0]['SHAP Value']

    print(f"The top feature is '{top_feature}' with a SHAP value of {top_shap_value}.")

    return AN, explainer, features, top_feature

# Example usage:
trained_model, explainer, features, top_feature = train_and_explain_model('/content/new_dataset_rf (2).csv', 'trained_model.h5')


Epoch 1/100
376/376 [==============================] - 4s 6ms/step - loss: 0.5002 - accuracy: 0.7498
Epoch 2/100
376/376 [==============================] - 2s 6ms/step - loss: 0.2730 - accuracy: 0.9124
Epoch 3/100
376/376 [==============================] - 2s 6ms/step - loss: 0.2093 - accuracy: 0.9322
Epoch 4/100
376/376 [==============================] - 3s 7ms/step - loss: 0.1825 - accuracy: 0.9422
Epoch 5/100
376/376 [==============================] - 1s 4ms/step - loss: 0.1604 - accuracy: 0.9488
Epoch 6/100
376/376 [==============================] - 1s 3ms/step - loss: 0.1431 - accuracy: 0.9548
Epoch 7/100
376/376 [==============================] - 1s 3ms/step - loss: 0.1353 - accuracy: 0.9555
Epoch 8/100
376/376 [==============================] - 1s 3ms/step - loss: 0.1241 - accuracy: 0.9608
Epoch 9/100
376/376 [==============================] - 1s 3ms/step - loss: 0.1218 - accuracy: 0.9601
Epoch 10/100
376/376 [==============================] - 1s 3ms/step - loss: 0.1148 - accura

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


94/94 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

      benign       0.98      0.99      0.99      1894
      malign       0.99      0.97      0.98      1114

    accuracy                           0.99      3008
   macro avg       0.99      0.98      0.98      3008
weighted avg       0.99      0.99      0.99      3008



PermutationExplainer explainer: 3009it [07:19,  6.76it/s]


The top feature is 'SEND_SMS' with a SHAP value of 0.050851745934393545.


In [44]:
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
from androguard.core.bytecodes.apk import APK

def extract_permissions(apk_path):
    try:
        # Load the APK file
        a = APK(apk_path)

        # Get the requested permissions
        permissions = a.get_permissions()

        # Remove the "android.permission." prefix from each permission
        permissions = [permission.replace("android.permission.", "") for permission in permissions]

        return permissions
    except Exception as e:
        print(f"Error: {e}")
        return None

def create_csv(apk_path, feature_names, csv_filename):
    extracted_permissions = extract_permissions(apk_path)

    if extracted_permissions:
        # Initialize a dictionary to store feature values (0 or 1)
        feature_values = {feature: 0 for feature in feature_names}

        # Check if each extracted permission is present in the feature names
        for permission in extracted_permissions:
            if permission in feature_names:
                feature_values[permission] = 1

        # Write the feature values to a CSV file
        with open(csv_filename, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=feature_names)
            writer.writeheader()
            writer.writerow(feature_values)
    else:
        print("Failed to extract permissions.")

def predict_malware(trained_model, features, app_features_path):
    # Load the CSV file containing the application features
    app_features_df = pd.read_csv(app_features_path)

    # Ensure that the input features are in the same order as the training features
    app_features = app_features_df[features]

    # Convert the features to a NumPy array
    app_features_array = np.array(app_features)

    # Use the trained model to predict
    predicted_probabilities = trained_model.predict(app_features_array)

    # Determine if the application is malware or not based on the prediction
    if predicted_probabilities[0] > 0.5:
        explanation = f"This application is predicted to be malware with a probability of {predicted_probabilities[0][0]}."
    else:
        explanation = f"This application is predicted to be benign with a probability of {1 - predicted_probabilities[0][0]}."

    # Print explanation
    print(explanation)

if __name__ == "__main__":
    # Path to the APK file
    apk_path = '/content/com.instagram.android_316.0.0.38.109-371611214_minAPI24(armeabi-v7a)(360,400,420,480dpi)_apkmirror.com.apk'

    # Path to the new.txt file
    new_txt_path = '/content/features.txt'

    # Path to the output CSV file
    output_csv_path = 'output.csv'

    # Path to the CSV file containing application features
    app_features_path = '/content/room.csv'

    # Read feature names from new.txt
    with open(new_txt_path, 'r') as f:
        features = f.read().splitlines()

    # Create CSV file for the given APK
    create_csv(apk_path, features, output_csv_path)

    # Load the trained model
    trained_model = tf.keras.models.load_model('trained_model.h5')

    # Predict malware using the created CSV file
    predict_malware(trained_model, features, app_features_path)



1/1 [==============================] - 0s 162ms/step
This application is predicted to be malware with a probability of 0.9999967813491821.


In [10]:
pip install androguard==3.3.5
